In [440]:
#load in libraries
library(ggplot2)
library(scales)
library(lubridate)
library(shiny)
library(shinythemes)
library(plyr)

ui <- fluidPage(
  
  theme = shinytheme("sandstone"),
  titlePanel("COVID-19: Tool for School Reopening"),
  
  navbarPage(
    "Explore",
    
    tabPanel(
      "Tool",
      sidebarLayout(
        sidebarPanel(
          width = 3,
          
          dateInput("tvsd", 
                    h5("teacher vaccination start date"), 
                    value = "2021-03-01"
          ),
          numericInput("tvw", 
                       h5("teacher vaccination window [*amount of time expected for all teachers who desire the vaccine to gain access to it]"), 
                       value = 14
          ),
          dateInput("sbsd", 
                    h5("first day of spring break"), 
                    value = "2021-03-20"
          ),
          dateInput("sbed", 
                    h5("last day of spring break"), 
                    value = "2021-04-04"
          ),
          dateInput("tbas", 
                    h5("teachers return to classroom"), 
                    value = "2021-04-05"
          ),
          dateInput("sbas", 
                    h5("students return to classroom"), 
                    value = "2021-04-12"
          ),
        ),
        
        mainPanel(
          plotOutput(
            "time.line", 
            height="600px", 
            width="100%"
          ),
          
          uiOutput(
            "details" 
          )
          
        )
      )
    ),
    
    
    
    tabPanel(
      "References",
      sidebarLayout(
        sidebarPanel(
          width = 3,
          h3("thanks for reopening safely!")
        ),
        
        mainPanel(
          h4("vaccine data sourced from ", a("here.", href = "https://www.kff.org/coronavirus-covid-19/issue-brief/what-does-a-multi-dose-series-mean-for-the-covid-19-vaccination-effort/")), 
          h4("repo with source code ", a("here.", href = "https://github.com/c-leber")),
          h4("more about the app's creator ", a("here.", href = "https://www.linkedin.com/in/christopheraleber/")),
          h4("viz inspiration ", a("here", href = "https://benalexkeen.com/creating-a-timeline-graphic-using-r-and-ggplot2/"), " and ", a("here.", href = "http://sape.inf.usi.ch/quick-reference/ggplot2/geom_rect"))
        )
      )
    )
    
  )
)

server <- function(input, output) {
  
    #timeline plot base
    tl.base <- ggplot() + 
        ggtitle('School Reopening Timeline') +
        ylim(-0.5, 10) +
        theme_classic() +
        scale_x_continuous(name="x") + 
        scale_y_continuous(name="y") +
        geom_hline(yintercept = 0, color = "black", size = 0.3) +
        theme(axis.line.y = element_blank(), 
          axis.text.y = element_blank(), 
          axis.title.x = element_blank(), 
          axis.title.y = element_blank(), 
          axis.ticks.y = element_blank(),
          axis.ticks.x = element_blank(),
          axis.line.x = element_blank(),
          legend.position = 'none',
          plot.title = element_text(size = 20, face = "bold")
            )
  
    buffer <- 7 #number of days
  
    output$time.line <- renderPlot({
    
        teacher.vaccination.start.date <- input$tvsd
        teacher.vaccination.window <- input$tvw

        spring.break.start.date <- input$sbsd
        spring.break.end.date <- input$sbed

        teachers.at.school.date <- input$tbas
        students.at.school.date <- input$sbas

        #generated from inputs
        teacher.vaccination.end.date <- teacher.vaccination.start.date + teacher.vaccination.window

        second.shot.moderna.start.date <- teacher.vaccination.start.date + 28
        second.shot.moderna.end.date <- teacher.vaccination.end.date + 28
        second.shot.pfizer.start.date <- teacher.vaccination.start.date + 21
        second.shot.pfizer.end.date <- teacher.vaccination.end.date + 21

        peak.effective.moderna.start.date <- teacher.vaccination.start.date + 28 + 14
        peak.effective.moderna.end.date <- teacher.vaccination.end.date + 28 + 14
        peak.effective.pfizer.start.date <- teacher.vaccination.start.date + 21 + 7
        peak.effective.pfizer.end.date <- teacher.vaccination.end.date + 21 + 7

        spring.break.quarantine.start.date <- spring.break.end.date
        spring.break.quarantine.end.date <- spring.break.end.date + 14

        start.dates <- c(teacher.vaccination.start.date, spring.break.start.date, second.shot.moderna.start.date, second.shot.pfizer.start.date, peak.effective.moderna.start.date, peak.effective.pfizer.start.date, spring.break.quarantine.start.date)
        end.dates <- c(teacher.vaccination.end.date, spring.break.end.date, second.shot.moderna.end.date, second.shot.pfizer.end.date, peak.effective.moderna.end.date, peak.effective.pfizer.end.date, spring.break.quarantine.end.date)
        all.dates <- c(start.dates, end.dates, teachers.at.school.date, students.at.school.date)
        
        #dataframe for plotting blocks that indicate date ranges
        rects <- data.frame(start = start.dates, end = end.dates, y1 = c(7, 1, 6, 4, 5, 3, 1), 
                            y2=c(8, 2, 7, 5, 6, 4, 2), 
                            t=c('1st dose', 'spring\nbreak', '2nd dose\n[Moderna]', '2nd dose\n[Pfizer]', 
                                'reach peak\neffectivity [Moderna]', 'reach peak\neffectivity [Pfizer]', 
                                'post spring\nbreak quaratine'), 
                            r=c('1st dose', 'spring\nbreak', '2nd dose\n[Moderna]', '2nd dose\n[Pfizer]', 
                                'reach peak\neffectivity [Moderna]', 'reach peak\neffectivity [Pfizer]', 
                                'post spring\nbreak quaratine'))

        date.range <- seq(min(all.dates) - buffer, max(all.dates) + buffer, by = 1)
        timeline.range <- data.frame(date.range)
        timeline.range$x.pos <- seq(1, nrow(timeline.range))

        rects$x1 <- mapvalues(as.character(rects$start), 
                              from=as.character(timeline.range$date.range), 
                              to=timeline.range$x.pos,
                              warn_missing = FALSE)
        rects$x1 <- as.numeric(rects$x1)

        rects$x2 <- mapvalues(as.character(rects$end), 
                              from=as.character(timeline.range$date.range), 
                              to=timeline.range$x.pos,
                              warn_missing = FALSE)
        rects$x2 <- as.numeric(rects$x2)

        #dataframe for plotting single events
        events <- data.frame(description <- c('teachers return\nto classroom', 'students return\nto classroom'))
        events$date <- c(teachers.at.school.date, students.at.school.date)
        events$x <- mapvalues(as.character(events$date), 
                              from = as.character(timeline.range$date.range), 
                              to = timeline.range$x.pos,
                              warn_missing = FALSE)
        events$x <- as.numeric(events$x)
        events$height <- c(10, 9)
        events$offset <- c(0.4, 0.4)

        plot <- tl.base +
          geom_rect(data = rects, mapping = aes(xmin = x1, xmax = x2, ymin=y1, ymax=y2, fill=t), color="black", alpha=0.5) +
          geom_text(data = rects, aes(x=x1+(x2-x1)/2, y=y1+(y2-y1)/2, label=r), size=4) +
          geom_text(data = timeline.range, aes(x = x.pos, y = 1, label = '')) +
          geom_segment(data = rects, aes(x = x1, y = y1, xend = x1, yend = 0), color = 'black', size = 0.1) +
          geom_point(data = rects, aes(x = x1, y = 0), size = 3) +
          geom_segment(data = rects, aes(x = x2, y = y1, xend = x2, yend = 0), color = 'black', size = 0.1) +
          geom_point(data = rects, aes(x = x2, y = 0), size = 3) +

          geom_segment(data = events, aes(x = x, y = height, yend = 0, xend = x), color = 'black', size = 0.1) +
          geom_point(data = events, aes(x = x, y = 0), size = 3) +
          geom_text(data = events, aes(x = x, y = height + offset, label = description), size = 4) +
          scale_x_continuous(breaks = seq(3, nrow(timeline.range), by = 7),
                             labels = mapvalues(seq(3, nrow(timeline.range), by = 7),
                                                from = as.character(timeline.range$x.pos),
                                                to = as.character(paste(month(timeline.range$date.range, label = TRUE), day(timeline.range$date.range))),
                                                warn_missing = FALSE)
          ) +
          theme(axis.text.x = element_text(angle = 90, vjust = 0, hjust=0, size = 12))
        
        #change plot title and color, depending on if the plan is deemed safe
        if ((peak.effective.moderna.end.date > students.at.school.date) | (spring.break.quarantine.end.date > students.at.school.date)) {
          plot +
            ggtitle('School Reopening Timeline - NOT SAFE') +
            theme(plot.title = element_text(color = "red"))
        } else {
          plot +
            ggtitle('School Reopening Timeline - SAFE') +
            theme(plot.title = element_text(color = "green4"))
    }
    
  })
  
  output$details <- renderUI({
    teacher.vaccination.start.date <- input$tvsd
    teacher.vaccination.window <- input$tvw
    
    spring.break.start.date <- input$sbsd
    spring.break.end.date <- input$sbed
    
    teachers.at.school.date <- input$tbas
    students.at.school.date <- input$sbas
    
    #generated from inputs
    teacher.vaccination.end.date <- teacher.vaccination.start.date + teacher.vaccination.window
    
    second.shot.moderna.start.date <- teacher.vaccination.start.date + 28
    second.shot.moderna.end.date <- teacher.vaccination.end.date + 28
    second.shot.pfizer.start.date <- teacher.vaccination.start.date + 21
    second.shot.pfizer.end.date <- teacher.vaccination.end.date + 21
    
    peak.effective.moderna.start.date <- teacher.vaccination.start.date + 28 + 14
    peak.effective.moderna.end.date <- teacher.vaccination.end.date + 28 + 14
    peak.effective.pfizer.start.date <- teacher.vaccination.start.date + 21 + 7
    peak.effective.pfizer.end.date <- teacher.vaccination.end.date + 21 + 7
    
    spring.break.quarantine.start.date <- spring.break.end.date
    spring.break.quarantine.end.date <- spring.break.end.date + 14
    
    #provide different details about the plan, depending on what conditions are or are not met
    if ((peak.effective.moderna.end.date > students.at.school.date) & (spring.break.quarantine.end.date > students.at.school.date)) {
      h4('Details: Your plan involves students returning to the classroom before teachers have had enough time to be effectively vaccinated and without completing a full quarantine following spring break. Therefore, the plan involves unnecessary risk, and so is deemed unsafe.')
    } else if (peak.effective.moderna.end.date > students.at.school.date){
      h4('Details: Your plan involves students returning to the classroom before teachers have had enough time to be effectively vaccinated. Therefore, the plan involves unnecessary risk, and so is deemed unsafe.')
    } else if (spring.break.quarantine.end.date > students.at.school.date){
      h4('Details: Your plan involves students returning to the classroom without completing a full quarantine following spring break. Therefore, the plan involves unnecessary risk, and so is deemed unsafe.')
    } else {
      h4('Details: Your plan allows for teachers to be effectively vaccinated and for the completion of a post spring break quarantine prior to students returning to the classroom. This plan limits unnecessary risk, and so is deemed safe.')
    }
  })
  
}

shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:4878

Scale for 'x' is already present. Adding another scale for 'x', which will
replace the existing scale.

